<a href="https://colab.research.google.com/github/soohyunme/exercise/blob/main/Code/02_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
cd /content/drive/MyDrive/4조/01.체육공모전/Code

/content/drive/.shortcut-targets-by-id/159Rw9eptrdiP8wihxnfPLcnRcwjx1x8n/4조/01.체육공모전/Code


# 모듈 로드

In [2]:
import random
import pandas as pd
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules


# 데이터 로드

In [3]:
df_origin = pd.read_csv('../Data/04.processed_data.csv',encoding='cp949')
group = pd.read_csv('../Data/user_group.csv',encoding='cp949')
ex_db = pd.read_csv('../Data/exercise_DB.csv',encoding='cp949')
ex_db['NEW_EX_NM'] = ex_db['EX_NM'].apply(lambda x : x.replace(' ',''))
ex_dict = dict(zip(list(ex_db['NEW_EX_NM']),list(ex_db['EX_NM'])))


In [4]:
user_db = df_origin[:-20]
sample_user = df_origin[-20:].iloc[:,:-3]

In [5]:
sample_user.to_csv('../Data/input.csv',encoding='cp949',index=False)


# 함수 정의

## 데이터프레임 변환 함수

In [6]:
# 나이를 연령대로 변환
def age_group(df):
  df['AGRDE_SE'] = df['TEST_AGE'].apply(lambda x : '10대' if x<20 else (\
                                                    '20대' if x<30 else (\
                                                      '30대' if x<40 else (\
                                                    '40대' if x<50 else (\
                                                    '50대' if x<60 else (\
                                                      '60대' if x<70 else (\
                                                    '70대 이상' )))))))
  return df

In [7]:
# 키와 몸무게를 통해 BMI 계산, BMI를 통해 비만 단계 변환
def bmi_group(df):
  bmi = pd.Series(round(df['WEIGHT'] / (df['HEIGHT']/100)**2,1),name='BMI')
  bmi_g = bmi.apply(lambda x : '3단계비만' if x>=35 else (\
                                                    '2단계비만' if x>=30 else (\
                                                      '1단계비만' if x>=25 else (\
                                                    '비만전단계' if x>=23 else (\
                                                    '정상' if x>=18.5 else '저체중')))))
  result = pd.concat([df,bmi_g],axis=1)
  return result

In [8]:
# Add user group column
def add_group(df):
  result = df.merge(group,how='inner')
  return result

In [9]:
def transform(df):
  result = age_group(df.copy())
  result = bmi_group(result)
  result = add_group(result)
  if '준비운동' in df.columns:
    result = result[['USER_ID', 'GROUP_ID', '준비운동', '본운동', '마무리운동']]
  else:
    result = result[['USER_ID', 'GROUP_ID']]
  return result

# 데이터셋 변환  
연령대, 운동강도, 성별, BMI 등급을 통한 그룹 변환

In [10]:
# User DB & sample user data transform
trans_db = transform(user_db)
trans_sample = transform(sample_user)

# exercise set df
set_df = pd.DataFrame(trans_db.drop('USER_ID',axis=1).value_counts()).reset_index().rename(columns={0:'COUNT'})

## 최빈 운동 세트 선정 함수

In [11]:
# input : exercise list / output : sparse matrix
def sparse_matrix(ex_list):
  te = TransactionEncoder()
  te_ary = te.fit(ex_list).transform(ex_list)
  sparse_matrix = pd.DataFrame(te_ary, columns=te.columns_)
  return sparse_matrix

In [12]:
def ranking(pre_df, main_df, finish_df, ex_list):
  df_list = [pre_df, main_df, finish_df] # 단계별 연관분석 결과값
  result_list = [] # 결과값을 리스트에 담아 하나씩 출력
  result_df = pd.DataFrame(columns=pre_df.columns) # 빈 DataFrame 생성
  for i in range(len(df_list)):
    tmp_df = df_list[i] # 운동 단계별 for문 pre, main, finish
    for each_ex in ex_list[i].split(','): # 각 단계별 운동들을 리스트로 분할
      each_ex = each_ex # 운동이름에 공백 제거
      each_result = tmp_df[tmp_df['antecedents']==frozenset({each_ex})].sort_values('support') # 해당하는 운동을 연관분석을 통해 지지도 순으로 정렬
      each_result['consequents'] = each_result['consequents'].apply(lambda x : list(x)[0]) # frozenset to string
      length = 3 if len(each_result) else len(each_result) # set length
      result_df = pd.concat([result_df,each_result.iloc[:length]]) # 연관성이 높은 운동이 3개 이상일 경우 3개만 저장
    result_list.append(result_df['consequents']) # 결과값 리스트에 추가
    result_df = None # 초기화
  return list(result_list[0]), list(result_list[1]), list(result_list[2]) # 준비, 본, 마무리 운동 연관성 높은 운동 리턴 / type : Series

In [13]:
# select top 1 exercise set from user info
# input : group_id / ouput : top 1 exercise set(pre, main, finish / Series)
def mode_set(g_id, min_support = 0.01, min_threshold=0.2):
  user_set = set_df[set_df['GROUP_ID']==g_id] # 유저가 해당하는 그룹의 처방 운동 세트 빈도
  max_ex_set = list(user_set[user_set['COUNT'] == user_set['COUNT'].max()].iloc[0,1:4]) # 가장 많이 처방된 운동 세트

  # 해당 그룹의 단계별 운동 리스트 생성
  pre_ex_all = list(user_set['준비운동'].apply(lambda x : x.split(','))) 
  main_ex_all = list(user_set['본운동'].apply(lambda x : x.split(',')))
  finish_ex_all = list(user_set['마무리운동'].apply(lambda x : x.split(',')))

  # 단계별 sparse matrix 생성
  pre_sparse = sparse_matrix(pre_ex_all)
  main_sparse = sparse_matrix(main_ex_all)
  finish_sparse = sparse_matrix(finish_ex_all)

  # 지지도가 0.01 이상인 운동 선택
  pre_itemsets = apriori(pre_sparse, min_support = min_support, use_colnames=True,max_len=2)
  main_itemsets = apriori(main_sparse, min_support = min_support, use_colnames=True,max_len=2)
  finish_itemsets = apriori(finish_sparse, min_support = min_support, use_colnames=True,max_len=2)

  # 그 중에서 신뢰도가 0.5 이상인 운동 선택
  pre_select = association_rules(pre_itemsets, metric="confidence", min_threshold=min_threshold)
  main_select = association_rules(main_itemsets, metric="confidence", min_threshold=min_threshold)
  finish_select = association_rules(finish_itemsets, metric="confidence", min_threshold=min_threshold)
  
  # consequents의 길이가 1인 것만 선택 -> 1개의 운동만 추천하도록
  pre_select = pre_select[pre_select['consequents'].apply(lambda x : len(x) == 1)]
  main_select = main_select[main_select['consequents'].apply(lambda x : len(x) == 1)] 
  finish_select = finish_select[finish_select['consequents'].apply(lambda x : len(x) == 1)]
  
  # ranking 
  pre_rank, main_rank, finish_rank = ranking(pre_select, main_select, finish_select, max_ex_set)

  # select exercise 
  a, b = select_exercise(pre_rank, main_rank, finish_rank, max_ex_set)

  return a, b

In [14]:
# select main routine
def select_exercise(pre_list, main_list, finish_list, max):
  main_reco_list = [] # 메인 추천 리스트
  sub_reco_list = [] # 추가 추천 리스트
  tmp_list = []
  asso_list = [pre_list,main_list,finish_list]
  tmp_set = set()
  random_num = random.randint(1, 100) # 난수 생성
  # Main recommend set
  for i, str in enumerate(max):
    ex_list = str.split(',') # 운동 세트에서 각 단계별 운동들
    reco_ex = ex_list[random_num %len(ex_list)] # 난수를 통해 임의의 1개 운동 선택
    main_reco_list.append(reco_ex) # 메인 추천 리스트에 선택된 운동 삽입

    tmp_set.update(ex_list) # 임시 리스트에 max 운동 리스트 삽입
    tmp_set.remove(reco_ex) # 선택된 운동 삭제 -> 단계별 최빈 세트 운동에서 메인 추천 운동 제거
    tmp_set.update(asso_list[i]) # 연관 분석을 통해 추천된 운동들 추가
    tmp_list.append(list(tmp_set)) # 해당 set을 리스트로 변환하여 tmp_list에 추가 -> [ [준,], [본,], [마,] ]
  for i, reco in enumerate(tmp_list):
    if (len(reco)) < 4:
      random_list = reco
    else:
      random_list = random.sample(reco, 4)
    sub_reco_list.append(random_list)
  return list(main_reco_list), list(sub_reco_list)

# 맞춤 운동 추천

In [15]:
main_reco = []
sub_reco = []
u_id_list = []
result_df = sample_user.copy()

In [16]:
for u_id, g_id in trans_sample[['USER_ID','GROUP_ID']].values:
  best, sub = mode_set(g_id) # 추천 세트 및 추가 추천 세트
  main_reco.append(list(best)) 
  sub_reco.append(sub)
  u_id_list.append(u_id)


In [17]:
reco_df = pd.DataFrame(main_reco,index=u_id_list,columns=['추천준비운동','추천본운동','추천마무리운동'])
sub_df = pd.DataFrame(sub_reco,index=u_id_list,columns=['추가준비운동','추가본운동','추가마무리운동'])
merge_df = pd.merge(reco_df,sub_df,left_index=True,right_index=True).reset_index().rename(columns={'index':'USER_ID'})
result_df = pd.merge(sample_user,merge_df)


In [18]:
# 결과 저장
result_df.to_csv('../Data/result.csv',encoding='cp949',index=False)
